## preProcessing

In [61]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D, Activation ,Dense ,Dropout , Flatten ,ConvLSTM2D
from keras.utils import np_utils
import matplotlib.pyplot as plt
from keras.optimizers import SGD,RMSprop,adam ,adadelta
import os ,cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.cross_validation import train_test_split
from keras import backend as K
K.set_image_dim_ordering('th')
from keras.preprocessing.image import ImageDataGenerator
from sklearn import preprocessing

C:\Users\DU357159\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\DU357159\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [66]:
path = "C://Users//DU357159//Documents//ENU_DATA//data"
data_list_directory = os.listdir(path)

In [67]:
img_rows = 128
img_col = 128
no_of_channel = 1
num_epochs =20
num_classes =2

In [68]:
img_data_list = []

In [69]:
for dataset in data_list_directory:
    img_path = os.listdir(path +"//"+dataset)
    for img in img_path:
        input_img = cv2.imread(path +"//"+dataset+"//"+img)
        input_img = cv2.cvtColor(input_img,cv2.COLOR_BGR2GRAY)
        input_img_resize = cv2.resize(input_img,(128,128))
        img_data_list.append(input_img_resize)
    

In [70]:
image_array = np.array(img_data_list)
image_array = image_array.astype('float32')

In [71]:
image_array = image_array.astype('float32')
image_array /= 255

In [72]:
image_array.shape

(150, 128, 128)

In [73]:
no_of_samples = image_array.shape[0]
labels = np.ones((no_of_samples) , dtype='int64')
labels[0:71] = 0
labels[71:] = 1
Y = np_utils.to_categorical(labels, num_classes)

In [74]:
x,y = shuffle(image_array ,Y ,random_state = 2)

In [75]:
X_train ,X_test ,y_train,y_test = train_test_split(x,y,test_size =0.2 ,random_state =4)
print(X_train.shape)
print(y_train.shape)
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

X_train = X_train.reshape(X_train.shape[0] , 1,img_rows ,img_col )
X_test = X_test.reshape(X_test.shape[0] , 1,img_rows ,img_col)
datagen.fit(X_train)

(120, 128, 128)
(120, 2)


## defining Model

In [76]:
print(X_train.shape)

(120, 1, 128, 128)


In [289]:
input_shape = (1,img_rows,img_col )
print(input_shape)

(1, 128, 128)


In [290]:
image_array.shape

(149, 128, 128)

In [297]:
model = Sequential()

In [298]:
model.add(Convolution2D(32 , (3,3) ,input_shape = input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [299]:
model.add(Convolution2D(64,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

In [300]:
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [301]:
model.compile(loss = 'categorical_crossentropy', optimizer='adadelta',metrics=["accuracy"])

In [302]:
#model.fit(X_train,y_tarin ,batch_size=16 ,epochs = 15 ,verbose =1 ,validation_data =(X_test,y_test))
model.fit_generator(datagen.flow(X_train, y_train, batch_size=16),epochs=15)

Epoch 1/15
8/8 [==============================] - 21s 3s/step - loss: 1.0586 - acc: 0.5320
Epoch 2/15
8/8 [==============================] - 21s 3s/step - loss: 0.5713 - acc: 0.7232
Epoch 3/15
8/8 [==============================] - 24s 3s/step - loss: 0.5147 - acc: 0.7952
Epoch 4/15
8/8 [==============================] - 23s 3s/step - loss: 0.5509 - acc: 0.7931
Epoch 5/15
8/8 [==============================] - 22s 3s/step - loss: 0.5473 - acc: 0.7447
Epoch 6/15
8/8 [==============================] - 24s 3s/step - loss: 0.5312 - acc: 0.7537
Epoch 7/15
8/8 [==============================] - 23s 3s/step - loss: 0.5396 - acc: 0.7773
Epoch 8/15
8/8 [==============================] - 21s 3s/step - loss: 0.4591 - acc: 0.8188
Epoch 9/15
8/8 [==============================] - 20s 3s/step - loss: 0.5104 - acc: 0.8088
Epoch 10/15
8/8 [==============================] - 19s 2s/step - loss: 0.4292 - acc: 0.8099
Epoch 11/15
8/8 [==============================] - 19s 2s/step - loss: 0.4242 - acc: 0.81

In [303]:
score = model.evaluate(X_test, y_test, verbose=0)
print('Test Loss:', score[0])
print('Test accuracy:', score[1])

Test Loss: 0.9461687207221985
Test accuracy: 0.5333333611488342


In [304]:
test_image = X_test[0:2]
print (test_image.shape)

print(model.predict(test_image))
print(model.predict_classes(test_image))
print(y_test[0:2])

(2, 1, 128, 128)
[[0.86155784 0.13844211]
 [0.8692552  0.13074487]]
[0 0]
[[1. 0.]
 [1. 0.]]


In [305]:
from sklearn.metrics import classification_report,confusion_matrix

In [306]:
Y_pred = model.predict(X_test)

y_pred = np.argmax(Y_pred, axis=1)
print(y_pred)
target_names = ['class 0(maps)', 'class 1(Sismic)']
print(classification_report(np.argmax(y_test,axis=1), y_pred,target_names=target_names))

print(confusion_matrix(np.argmax(y_test,axis=1), y_pred))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
                 precision    recall  f1-score   support

  class 0(maps)       0.53      1.00      0.70        16
class 1(Sismic)       0.00      0.00      0.00        14

    avg / total       0.28      0.53      0.37        30

[[16  0]
 [14  0]]


C:\Users\DU357159\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
